# NYC Ride Share Dateset:

## Purpose
1. take raw parquet files from NYC Gov website and
2. zip them into one large parquet file
3. change columns to friendly names
4. Drop unneede columns for analysis
5. Assign custom measures
6. save as a new parquet file
7. all subsequent testing of the ML model will be run through this ETL process.

In [1]:
#library imports
import dask.dataframe as dd
import os
import time

start = time.time()

## Data location and merging
1. Sets Folder path to source data
2. Read in all parquet files in the source folder
3. merge them into a singular df

In [2]:
#source folder for parquet storage
source_data_folder = 'source_data'

#reads in all parquet files in source_data folder
df = dd.read_parquet(source_data_folder, engine='pyarrow')

## Column Rename, Column Drops
1. Rename Columns to 'Friendly Names'
2. Sets Columns to Drop (ride share flags)

In [3]:
#mapping of column names
column_mapping = {
    'hvfhs_license_num': 'business',
    'dispatching_base_num': 'd1',
    'originating_base_num': 'd2',
    'request_datetime': 'request_time',
    'on_scene_datetime': 'on_scene_time',
    'pickup_datetime': 'pickup_time',
    'dropoff_datetime': 'dropoff_time',
    'PULocationID': 'pickup_location',
    'DOLocationID': 'dropoff_location',
    'trip_miles': 'trip_length',
    'trip_time': 'trip_time_seconds',
    'base_passenger_fare': 'base_passenger_fare',
    'tolls': 'tolls',
    'bcf': 'bcf',
    'sales_tax': 'sales_tax',
    'congestion_surcharge': 'congestion_surcharge',
    'airport_fee': 'airport_fee',
    'tips': 'tips',
    'driver_pay': 'driver_pay',
    'shared_request_flag': 'd3',
    'shared_match_flag': 'd4',
    'access_a_ride_flag': 'd5',
    'wav_request_flag': 'd6',
    'wav_match_flag': 'd7',
    }

#columns to drop
columns_to_drop = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7']


#rename
df = df.rename(columns=column_mapping)

#drop columns
df = df.drop(columns=columns_to_drop)


## Remapping of values & Custom Measures
1. Likely Most Resource Intensive will add timing function for eventual port to cudf

In [4]:
#business mapping
replacements = {
    'HV0002': 'Juno',
    'HV0003': 'Uber',
    'HV0004': 'Via',
    'HV0005': 'Lyft'
}

#replace values in business column
df['business'] = df['business'].map(replacements).fillna(df['business'])

### Time Measures

1. Request to Dropoff Time: The duration between the request time and the dropoff time : 'Total Time from Passenger's Perspective'
2. Request to Pickup Time: The duration between the request time and the pickup time. : 'The time it took from a request to a pickup'
3. Total Ride Time: The duration between the pickup and dropoff time. : 'Total Time Spent in the car'
4. On Scene to Pickup Time: The duration between the on-scene time and the pickup time. This can be useful to measure how long it takes for a passenger to board after the vehicle arrives. : 'How long did the driver wait?'
5. On Scene to Dropoff Time: The duration between the on-scene time and the dropoff time. This gives a sense of the total time commitment from the driver's perspective once they've arrived at the scene. : 'How much time did the driver invest in this one passenger'

In [5]:
#time measures

def time_diff(df, col1, col2):
    return df[col1] - df[col2]

times = {
    'request_to_dropoff': ['dropoff_time', 'request_time'],
    'request_to_pickup': ['pickup_time', 'request_time'],
    'total_ride_time': ['dropoff_time', 'pickup_time'],
    'on_scene_to_pickup': ['pickup_time', 'on_scene_time'],
    'on_scene_to_dropoff': ['dropoff_time', 'on_scene_time'] 
}

for key, cols in times.items():
    df[key] = time_diff(df, cols[0], cols[1])


def time_cat(df):
    hour = df['request_time'].dt.hour
    df['time_of_day'] = 'night'  # Default value
    df.loc[(hour >= 6) & (hour <= 11), 'time_of_day'] = 'morning'
    df.loc[(hour >= 12) & (hour <= 16), 'time_of_day'] = 'afternoon'
    df.loc[(hour >= 17) & (hour <= 19), 'time_of_day'] = 'evening'
    return df

# Apply the function to the DataFrame using map_partitions
df = df.map_partitions(time_cat)

df['date'] = df['request_time'].dt.date 
df['hour_of_day'] = df['request_time'].dt.hour
df['day_of_week'] = df['request_time'].dt.dayofweek
df['week_of_year'] = df['request_time'].dt.isocalendar().week
df['month_of_year'] = df['request_time'].dt.month

### Pay Measures
In this section, we calculate various financial metrics related to rideshare journeys. These metrics provide insights into the earnings and costs associated with each trip from both the passenger's and driver's perspective.

1. Passenger Fare: Total amount paid by the passenger for the ride. This includes the base fare, tips, tolls, taxes, surcharges, and any airport fees.
2. 
Driver Total Pay: The amount earned by the driver, comprising their pay and tips
3. Rideshare Profit: The difference between what the passenger pays and the amount received by the driver. It represents the company's earnings from the trip.|
4. Hourly Rate of Driver: The driver's earnings per hour, calculated using the on-scene time until the dropoff. This metric gives an idea of the driver's earnings efficiency.
5. Dollars Per Mile: This measure calculates the driver's earnings per mile driven, providing insight into the profitability of trips based on distance.cnce.

In [6]:
#what the passenger actually paid to take the ride
df['passenger_fare'] = df['base_passenger_fare'] + df['tips'] + df['tolls'] + df['sales_tax'] + df['congestion_surcharge'] + df['airport_fee']

#what the driver actually recieved
df['driver_total_pay'] = df['driver_pay'] + df['tips']

#rideshare profit
df['rideshare_profit'] = df['passenger_fare'] - df['driver_pay']

#calculate hourly rate of drive
#time delta to seconds
df['on_scene_seconds'] = df['on_scene_to_dropoff'].dt.total_seconds().astype(float)

#calculate on scene hours 
df['on_scene_hours'] = df['on_scene_seconds'] / 3600 

#hourly rate of driver
df['hourly_rate'] = df['driver_total_pay'] / df['on_scene_hours']

#pay of dollars per mile driven
df['dollars_per_mile'] = df['driver_total_pay'] / df['trip_length']

## Removal of Excess Columns and Export

In [7]:
# List of columns to remove
columns_to_remove = [
    'request_time', 'on_scene_time', 'pickup_time', 'dropoff_time',
    'trip_time_seconds', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
    'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
    'on_scene_seconds', 'day_of_week', 'on_scene_hours'
]

# Dropping the columns from the DataFrame
df = df.drop(columns=columns_to_remove)

In [8]:
# Force compute
final_df = df.compute()
final_df.to_parquet('rideshare_analysis.parquet')
print('data saved!')

data saved!


In [9]:
end = time.time()
print(f"Operation took: {end - start:.3f} seconds")

print('Number of rows:', len(final_df))
print('Number of columns:', len(final_df.columns))


Operation took: 25.317 seconds
Number of rows: 14751591
Number of columns: 20


In [10]:
print(final_df.dtypes)

business                        object
pickup_location                  int64
dropoff_location                 int64
trip_length                    float64
request_to_dropoff     timedelta64[ns]
request_to_pickup      timedelta64[ns]
total_ride_time        timedelta64[ns]
on_scene_to_pickup     timedelta64[ns]
on_scene_to_dropoff    timedelta64[ns]
time_of_day                     object
date                            object
hour_of_day                      int32
day_of_week                      int32
week_of_year                    UInt32
month_of_year                    int32
passenger_fare                 float64
driver_total_pay               float64
rideshare_profit               float64
hourly_rate                    float64
dollars_per_mile               float64
dtype: object


In [11]:
print(final_df.head())

  business  pickup_location  dropoff_location  trip_length request_to_dropoff  \
0     Uber              170               161         1.18    0 days 00:12:57   
1     Uber              237               161         0.82    0 days 00:10:45   
2     Uber              237               161         1.18    0 days 00:23:39   
3     Uber              262               229         1.65    0 days 00:07:29   
4     Uber              229               141         1.65    0 days 00:09:57   

  request_to_pickup total_ride_time on_scene_to_pickup on_scene_to_dropoff  \
0   0 days 00:01:53 0 days 00:11:04    0 days 00:01:44     0 days 00:12:48   
1   0 days 00:03:05 0 days 00:07:40    0 days 00:00:24     0 days 00:08:04   
2   0 days 00:13:44 0 days 00:09:55    0 days 00:00:00     0 days 00:09:55   
3   0 days 00:02:26 0 days 00:05:03    0 days 00:00:54     0 days 00:05:57   
4   0 days 00:02:16 0 days 00:07:41    0 days 00:02:00     0 days 00:09:41   

  time_of_day        date  hour_of_day  day_